In [ ]:
%load_ext autoreload
%autoreload 2

#import tensorflow as tf
#tf.config.list_physical_devices('GPU')
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import datetime
import json
import matplotlib.pyplot as plt
import os
import sys
import tensorflow as tf
from tensorflow.keras import layers

from twinnet_tools.tnconfig import ProjectConfig
from twinnet_tools.tnmodel import TNToolsDistanceLayer
from twinnet_tools.tnmodel import TNToolsModel
from twinnet_tools.tntraining import TNToolsTrainingImages, TNToolsTrainingPaths

os.environ["OPENCV_LOG_LEVEL"]="FATAL"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

config = ProjectConfig("twinnet_config")

Make tool instances

In [ ]:
img_height, img_width = 224, 224 
it = TNToolsTrainingImages(img_height, 280, img_width, 280)
pg = TNToolsTrainingPaths(img_height, 280, img_width, 280, pattern_glob_files="*.tif")

image_count = 1000000

batchSize = 256
prefetch = 8
buffer_size=1024 #round(image_count * 1.2)

outFolder = '../models/model1/'

# Step 1: Dataset

In [ ]:
#list_paths_datasets_directories = ["./dir_dataset_1",
#                                   "./dir_dataset_2"]
list_paths_datasets_directories1 = ["../trainData/zfish_285C"
                                   ]

list_paths_jsons_exclude1 = [str(i) + '/EMBRYOS_TO_EXCLUDE.json' for i in list_paths_datasets_directories1]

print(list_paths_datasets_directories1[0])

In [ ]:
paths_anchor, paths_positive, paths_negative = pg.procedure_paths_binary(list_paths_datasets_directories1,
                                                                         list_paths_jsons_exclude1,
                                                                         image_count)

In [ ]:
paths_anchor2, paths_positive2, paths_negative2 = pg.procedure_paths_binary(list_paths_datasets_directories1,
                                                                            list_paths_jsons_exclude1,
                                                                            image_count)

In [ ]:
paths_anchor3, paths_positive3, paths_negative3 = pg.procedure_paths_binary(list_paths_datasets_directories1,
                                                                            list_paths_jsons_exclude1,
                                                                            image_count)

In [ ]:
dataset_anchor = tf.data.Dataset.from_tensor_slices(paths_anchor)
dataset_positive = tf.data.Dataset.from_tensor_slices(paths_positive)
dataset_negative = tf.data.Dataset.from_tensor_slices(paths_negative)

dataset = tf.data.Dataset.zip((dataset_anchor, dataset_positive, dataset_negative))
dataset = dataset.shuffle(buffer_size=buffer_size)
dataset = dataset.map(it.parse_triple_fn)

train_dataset = dataset.take(round(image_count * 0.8))
val_dataset = dataset.skip(round(image_count * 0.8))

train_dataset = train_dataset.batch(batchSize, drop_remainder=False)
train_dataset = train_dataset.prefetch(prefetch)

val_dataset = val_dataset.batch(batchSize, drop_remainder=False)
val_dataset = val_dataset.prefetch(prefetch)

In [ ]:
dataset_anchor2 = tf.data.Dataset.from_tensor_slices(paths_anchor2)
dataset_positive2 = tf.data.Dataset.from_tensor_slices(paths_positive2)
dataset_negative2 = tf.data.Dataset.from_tensor_slices(paths_negative2)

dataset2 = tf.data.Dataset.zip((dataset_anchor2, dataset_positive2, dataset_negative2))
dataset2 = dataset2.shuffle(buffer_size=buffer_size)
dataset2 = dataset2.map(it.augment_triple_one)

train_dataset2 = dataset2.take(round(image_count * 0.8))
val_dataset2 = dataset2.skip(round(image_count * 0.8))

train_dataset2 = train_dataset2.batch(batchSize, drop_remainder=False)
train_dataset2 = train_dataset2.prefetch(prefetch)

val_dataset2 = val_dataset2.batch(batchSize, drop_remainder=False)
val_dataset2 = val_dataset2.prefetch(prefetch)


In [ ]:
dataset_anchor3 = tf.data.Dataset.from_tensor_slices(paths_anchor3)
dataset_positive3 = tf.data.Dataset.from_tensor_slices(paths_positive3)
dataset_negative3 = tf.data.Dataset.from_tensor_slices(paths_negative3)

dataset3 = tf.data.Dataset.zip((dataset_anchor3, dataset_positive3, dataset_negative3))
dataset3 = dataset3.shuffle(buffer_size=buffer_size)
dataset3 = dataset3.map(it.augment_triple_two)

train_dataset3 = dataset3.take(round(image_count * 0.8))
val_dataset3 = dataset3.skip(round(image_count * 0.8))

train_dataset3 = train_dataset3.batch(batchSize, drop_remainder=False)
train_dataset3 = train_dataset3.prefetch(prefetch)

val_dataset3 = val_dataset3.batch(batchSize, drop_remainder=False)
val_dataset3 = val_dataset3.prefetch(prefetch)

In [ ]:
def visualize(anchor, positive, negative):
    """Visualize a few triplets from the supplied batches."""

    def show(ax, image):
        ax.imshow(image / 255, cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    fig = plt.figure(figsize=(9, 9))

    axs = fig.subplots(4, 3)
    for i in range(4):
        show(axs[i, 0], anchor[i])
        show(axs[i, 1], positive[i])
        show(axs[i, 2], negative[i])


visualize(*list(train_dataset.take(1).as_numpy_iterator())[0])

# Step 2: Model

Define model

In [ ]:
base_cnn = tf.keras.applications.resnet.ResNet50(
    weights="imagenet", input_shape=(img_height, img_width) + (3,), include_top=False
)

flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

embedding = tf.keras.models.Model(base_cnn.input, output, name="Embedding")

trainable = False
for layer in base_cnn.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

In [ ]:
anchor_input = tf.keras.layers.Input(name="anchor", shape=(img_height, img_width) + (3,))
positive_input = tf.keras.layers.Input(name="positive", shape=(img_height, img_width) + (3,))
negative_input = tf.keras.layers.Input(name="negative", shape=(img_height, img_width) + (3,))

distances = TNToolsDistanceLayer()(
    embedding(tf.keras.applications.resnet.preprocess_input(anchor_input)),
    embedding(tf.keras.applications.resnet.preprocess_input(positive_input)),
    embedding(tf.keras.applications.resnet.preprocess_input(negative_input)),
)

twin_network = tf.keras.models.Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

twin_network_model = TNToolsModel(twin_network)

In [ ]:
twin_network.summary()

Define callbacks

In [ ]:
checkpoint_filepath = outFolder+'/checkpoints/'

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5,
)


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)


Run training:
- 6 runs, 2 with a different dataset
- 1000000 image triplets per dataset
- 10 epochs per dataset

In [ ]:
twin_network_model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), metrics=['accuracy'])


In [ ]:
log_dir = outFolder+"logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = twin_network_model.fit(train_dataset, epochs=10, callbacks=[early_stopping, model_checkpoint_callback, tensorboard_callback], validation_data=val_dataset)

In [ ]:
twin_network_model.compute_output_shape(input_shape=((None, 224,224,3), (None, 224,224,3),(None, 224,224,3)))

In [ ]:
twin_network_model.save(outFolder+'/dir_dst_model_epochs_10/')
embedding.save_weights(outFolder+'/dir_dst_model_epochs_10/')

In [ ]:
log_dir = outFolder+"logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history2 = twin_network_model.fit(train_dataset2, epochs=20, initial_epoch=10, callbacks=[early_stopping, model_checkpoint_callback, tensorboard_callback], validation_data=val_dataset2)

In [ ]:
twin_network_model.save(outFolder+'/dir_dst_model_epochs_20/')
embedding.save_weights(outFolder+'/dir_dst_model_epochs_20/')

In [ ]:
log_dir = outFolder+"logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history3 = twin_network_model.fit(train_dataset3, epochs=30, initial_epoch=20, callbacks=[early_stopping, model_checkpoint_callback,tensorboard_callback], validation_data=val_dataset3)

In [ ]:
twin_network_model.save(outFolder+'/dir_dst_model_epochs_30/')
embedding.save_weights(outFolder+'/dir_dst_model_epochs_30/')

In [ ]:
log_dir = outFolder+"logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history4 = twin_network_model.fit(train_dataset3, epochs=40, initial_epoch=30, callbacks=[early_stopping, model_checkpoint_callback,tensorboard_callback], validation_data=val_dataset3)
twin_network_model.save(outFolder+'/dir_dst_model_epochs_40/')
embedding.save_weights(outFolder+'/dir_dst_model_epochs_40/')
